In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import talib as ta
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import datetime
import pandas as pd

In [ ]:
!pip install backtesting 
!pip install ta-lib-binary 
!pip install yfinance 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# ticker and the start and end dates for testing
ticker =  input("Enter ticker: ")
start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2023, 1, 1)

Enter ticker: ^GDAXI


In [ ]:
df=yf.download( ticker , start=start,end=end)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,7689.459961,7789.939941,7689.459961,7778.779785,7778.779785,140345500
2013-01-03,7762.129883,7777.020020,7737.979980,7756.439941,7756.439941,86174100
2013-01-04,7740.990234,7779.669922,7727.549805,7776.370117,7776.370117,101577000
2013-01-07,7760.009766,7767.549805,7713.419922,7732.660156,7732.660156,151695400
2013-01-08,7710.700195,7754.379883,7683.479980,7695.830078,7695.830078,130425500
...,...,...,...,...,...,...
2022-12-23,13945.589844,14000.679688,13874.500000,13940.929688,13940.929688,28738700
2022-12-27,14047.419922,14063.139648,13966.349609,13995.099609,13995.099609,22975000
2022-12-28,14013.719727,14018.469727,13914.620117,13925.599609,13925.599609,27583800


In [ ]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [ ]:
class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
%%script echo

    def next(self):
        if (self.sma1[-2] < self.sma2[-2] and
                self.sma1[-1] > self.sma2[-1]):
            self.position.close()
            self.buy()

        elif (self.sma1[-2] > self.sma2[-2] and   
              self.sma1[-1] < self.sma2[-1]):
            self.position.close()
            self.sell()

In [ ]:
from backtesting import Backtest

bt = Backtest(df, SmaCross, cash=100_000, commission=.001)
stats = bt.run()

OPTIMIZE 


In [ ]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Return [%]',
                    constraint=lambda param: param.n1 < param.n2)
print(stats)

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Start                     2013-01-02 00:00:00
End                       2022-12-30 00:00:00
Duration                   3649 days 00:00:00
Exposure Time [%]                   98.064771
Equity Final [$]                110378.057957
Equity Peak [$]                 154334.829422
Return [%]                          10.378058
Buy & Hold Return [%]               78.994524
Return (Ann.) [%]                    0.987577
Volatility (Ann.) [%]               17.429522
Sharpe Ratio                         0.056661
Sortino Ratio                        0.082314
Calmar Ratio                         0.030289
Max. Drawdown [%]                  -32.604828
Avg. Drawdown [%]                   -5.737264
Max. Drawdown Duration     1503 days 00:00:00
Avg. Drawdown Duration      111 days 00:00:00
# Trades                                   70
Win Rate [%]                        32.857143
Best Trade [%]                      23.274076
Worst Trade [%]                    -11.250268
Avg. Trade [%]                    

In [ ]:
stats._strategy

<Strategy SmaCross(n1=20,n2=35)>

In [ ]:
bt.plot()


Row(id='9309', ...)

In [ ]:
stats['_trades']  # Contains individual trade data

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-16,66,84,6243.550005,6451.299805,-3323.996797,-0.033274,2013-04-08,2013-05-02,24 days
1,14,84,110,6457.751104,6400.500000,-801.515463,-0.008865,2013-05-02,2013-06-11,40 days
2,-14,110,134,6394.099500,6544.899902,-2111.205633,-0.023584,2013-06-11,2013-07-15,34 days
3,14,134,157,6551.444802,6587.399902,503.371401,0.005488,2013-07-15,2013-08-15,31 days
4,-14,157,178,6580.812502,6583.799805,-41.822231,-0.000454,2013-08-15,2013-09-16,32 days
...,...,...,...,...,...,...,...,...,...,...
116,-6,2390,2409,7009.283505,7039.799805,-183.097799,-0.004354,2022-06-20,2022-07-15,25 days
117,6,2409,2444,7046.839604,7281.200195,1406.163545,0.033258,2022-07-15,2022-09-05,52 days
118,-6,2444,2485,7273.918995,7186.200195,526.312799,0.012059,2022-09-05,2022-11-02,58 days
119,6,2485,2522,7193.386396,7469.299805,1655.480455,0.038357,2022-11-02,2022-12-23,51 days
